In [1]:
import numpy as np
import george
from george import kernels
from scipy.optimize import minimize
from time import time
import argparse
import pickle
import matplotlib.pyplot as plt

In [2]:
conv=np.load("./etgrid/3962_cobyla_convergence.npy")
conv.shape

(740, 16, 16)

In [3]:
param_names = ["Tstar","logL_star","logM_disk","logR_disk","h_0","logR_in",\
          "log gamma","bix","log logM_env","logR_env","f_cav","ksi","loga_max","p","biy"]

np.set_printoptions(suppress=True)
z=np.sqrt(np.exp(conv[0][0]))
n=np.sqrt(np.exp(conv[738][0]))

for i in range(len(param_names)):
    print(param_names[i], np.round(z[i+1],3), np.round(n[i+1],3))

Tstar 10000.0 10842.228
logL_star 1.0 0.378
logM_disk 1.0 0.577
logR_disk 0.8 0.441
h_0 0.08 0.076
logR_in 0.5 0.394
log gamma 0.3 0.061
bix 0.3 0.154
log logM_env 0.2 0.197
logR_env 0.2 0.07
f_cav 0.25 0.068
ksi 0.25 0.08
loga_max 2.0 0.693
p 3.0 5.53
biy 1.5 2.353


In [5]:
coords=np.load("./etgrid/3962_coords.npy")
eigenseds=np.load("./etgrid/3962_eigenseds.npy")
weights=np.load("./etgrid/3962_weights.npy")
pcamean=np.load("./etgrid/3962_mean.npy")
sedsflat=np.load("./etgrid/sedsflat.npy")

In [7]:
yerrs=[]
for i in range(16):
    yerrs.append([x*0.01 for x in weights[i]])

In [8]:
kernel = 16*kernels.ExpSquaredKernel(15**2,ndim=15,axes=0)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=1)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=2)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=3)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=4)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=5)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=6)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=7)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=8)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=9)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=10)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=11)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=12)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=13)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=14) 
blankhodlr=george.GP(kernel,solver=george.HODLRSolver)

In [9]:
def pred(pred_coords,fit):
    gp=blankhodlr
    preds=[]
    if "cobyla" in name:
        hyperparams=np.transpose(np.array(fit).reshape(16,16))
    else:
        hyperparams=np.array(fit).reshape(16,16)

    for i in range(len(weights)):  # same covfunc for each weight and the sample mean
        gp.set_parameter_vector(hyperparams[i])
        gp.compute(coords,yerrs[i])
        pred, pred_var = gp.predict(weights[i], pred_coords, return_var=True)
        preds.append(pred)
    reconst_SEDs=[]
    for i in range(len(pred_coords)):
        reconst=np.dot(np.array(preds)[:,i][0:15],eigenseds[0:15]) + pcamean + np.array(preds)[:,i][15]
        reconst_SEDs.append(reconst)
    return reconst_SEDs, preds